Models Used :-

We used the Mask RCNN model and trained it on our datset for 5 epochs.
We used a Feedforward neural network and trained it for 7 epochs.

Finetuning:-

For the Finetuning of classification model we used droputs,callbacks, normalization, autotune of the training and validation dataset

Pipelines:-

1.Mask RCNN-for health monitor segmentation
2.Classification model to classify the type of monitor
3.We used Tesseract OCR for digit extraction.

**Installing and unzipping the neccesary repositories**

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
# Check if package has been installed
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
# Upgrade Tensorflow
!pip install --upgrade tensorflow==2.5.0
!wget https://pysource.com/extra_files/maskrcnn_colab_demo_commit_17.zip
!unzip maskrcnn_colab_demo_commit_17.zip
import sys
sys.path.append("/content/maskrcnn_colab/mrcnn_demo")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

--2023-02-07 17:32:15--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430460776 (411M) [application/x-deb]
Saving to: ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’

libcudnn8_8.1.0.77- 100%[===================>] 410.52M   200MB/s    in 2.0s    

2023-02-07 17:32:17 (200 MB/s) - ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’ saved [430460776/430460776]

dpkg: warning: downgrading libcudnn8 from 8.1.1.33-1+cuda11.2 to 8.1.0.77-1+cuda11.2
(Reading database ... 129496 files and directories currently installed.)
Preparing to unpack libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+cuda11.2) over (8.1.1.33-1+cuda11.2) ...
Setting up libcudnn8 (8.1.0.77-1+c

/content/maskrcnn_colab/mrcnn_demo/model.py:2378: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if os.name is 'nt':


... done downloading pretrained model!


**Mounting the Drive**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
#uncomment if mounting the drive

Mounted at /content/drive


**Models**

In [ ]:
model1="" ##mention the mnodel path explicitly
model2=""##mention the mnodel path explicitly

**Inference Function**

In [ ]:
def inference(image_path):
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  result = {}

  ### put your code here
  import os
  from os import listdir
  import cv2
  import pandas as pd
  import pytesseract
  import csv
  import json
  import os
  import numpy as np
  from google.colab.patches import cv2_imshow

  # check if the image ends with png or jpg or jpeg
  img_n=cv2.imread(image_path)
  r = model1.detect([img_n])[0]
  colors = random_colors(1)
  object_count = len(r["class_ids"])
  mask = r["masks"][:, :, 0]
  colors = random_colors(1)
  contours = get_mask_contours(mask)
  bbx=cv2.boundingRect(contours[0])
  img_cropped=img_n[bbx[1]:bbx[1]+bbx[3],bbx[0]:bbx[0]+bbx[2]]
  cv2_imshow(img_cropped)
  test_img =img_cropped
  img = np.expand_dims(test_img, axis = 0)
  prediction=model2.predict(img)
  score = tf.nn.softmax(prediction)
  print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))


  def extract_digits(cropped):

    #pre-process
    cropped = cv2.resize(cropped, (620, 350))
    
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # cv2.imshow('processed', thresh)
    # cv2.waitKey(0)
    
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    kernel = np.ones((3,3),np.uint8)
    threshold = cv2.morphologyEx(opening, cv2.MORPH_OPEN, kernel)
    
    
    # Run OCR on the image
    text = pytesseract.image_to_string(threshold, config='--psm 6')
    # print(text)
    
    # Extract the digits from the OCR output
    digits = [char for char in text if char.isdigit()]
    
    # Return the extracted digits as a string
    return "".join(digits), text
        
        

        
  def use_ocr(bbox):
    output = {}
    xmin, ymin, xmax, ymax= bbox

    bbox= [xmin, ymin, xmax, ymax]
    #print(bbox)
    cropped = img[int(ymin):int(ymax), int(xmin):int(xmax)]

    #print("Extracted digits:", "".join(digits))
    #text= pytesseract.image_to_string(cropped, config='--psm 7 -c tessedit_char_whitelist=0123456789.%')

    # Extract the digits from the cropped image
    digits, text = extract_digits(cropped)
    # print(column_names[i], digits)

    # Store the extracted digits in the results dictionary
    return digits, text

    #         post_process(output)
    
  classif=class_names[np.argmax(score)]
  output={}
  if classif=="BPL-EliteView-EV10-B_Meditec-England-A":
    hr= [0,0,670,323] #left upper
    spo2= [0,329,663,391+663] # left lower
    rr=[675,0,605+675,328]# right upper
    clubbed =[670,335,610+670,385+335] #right lower
    v1,t= use_ocr(hr); output["HR"]=v1
    v2,t= use_ocr(spo2); output["SPO2"]=v2
    v3,t3= use_ocr(rr)
    v4,t4= use_ocr(clubbed)
    if(len(t3)>=4):
        getValues_1(output, v3, t3)
        output["RR"]=v4
    else:
        getValues_1(output, v4, t4)
        output["RR"]=v3
    
  elif classif=="BPL-EliteView-EV100-C":
    hr_bp=[865,38,415+865,205+38]
    spo2_rr=[859,297,370+859,254+297]
    v1,t= use_ocr(hr_bp)
    v2,t1= use_ocr(spo2_rr)
    print(v1, v2)
    output["HR"]=int(v1[0:2])
    output["MAP"]=int(v1)%100
    
  elif classif=="BPL-Ultima-PrimeD-A":
    hr=[815,0,465+815,258]
    spo2=[815,272,465+815,226+272]
    rr=[815,512,465+815,207+512]
    bp=[0,513,806,207+513]
    v1,t= use_ocr(hr)
    v2,t= use_ocr(spo2)
    v3,t= use_ocr(rr)
    v4,t= use_ocr(bp)
  #     print(v1, v2)
    output["HR"]=v1
    output["SPO2"]=v2
    output["RR"]=v3
    output["MAP"]=int(v4)%100
    output["DBP"]=int(v4/100)%100

    
  elif classif=="Nihon-Kohden-lifescope-A":
    hr=[0,39,476,232+39]
    spo2=[0,465,473,139+465]
    rr=[0,610,471,104+610]
    bp=[0,278,477,181+278]
    v1,t= use_ocr(hr)
    v2,t= use_ocr(spo2)
    v3,t= use_ocr(rr)
    v4,t= use_ocr(bp)
  #     print(v1, v2)
    output["HR"]=v1
    output["SPO2"]=v2
    output["RR"]=v3
    output["MAP"]=int(v4)%100
    output["DBP"]=(int(v4)/100)%100


  result=output

  return result

**Calling the Inference Function**

In [ ]:
inference("")##Provide the image path